# Place cell analyisis

In [16]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [17]:
import sys
sys.path.append('../src') # Add src folder to path.
import place_cells as pc # Import my place_cell functions from /src.

In [18]:
# load data

# Data path and name for animal 1
animal1_data_folder = Path(r"Z:\davide\2p_data\441406_fiano")
animal1 = '441406_fiano'

# Data path and name for animal 2
animal2_data_folder = Path(r"Z:\davide\2p_data\441394_ribolla")
animal2 = '441394_ribolla'

# List of data_paths and animal names
animal_folders = [animal1_data_folder, animal2_data_folder]
animal_names = [animal1, animal2]

# Enable plot saving and where to save
save_plots = True
output_folder = Path('./imgs')
output_folder.mkdir(exist_ok=True,parents=True)

# List of session dates for animal 1
animal1_session_dates = ['20230309', '20230317', '20230323', '20230328', '20230331']

# List of session dates for animal 2
animal2_session_dates = ['20230315', '20230324', '20230331', '20230404', '20230405']

# List of session dates for all animals
all_session_dates = [animal1_session_dates, animal2_session_dates]

# Create an empty DataFrame for the place cells
results_df = pd.DataFrame(columns=['Animal', 'Session Date', 'Environment', 'Number of Place Cells', 'Fraction'])

In [19]:
def process_sessions(animal_folder, animal_name, session_dates, results_df):

    # Loop over session dates
    for date in session_dates:
        # Check if the current item is a .zip file
        if date.endswith('.zip'):
            continue

        data_path = animal_folder.joinpath(date)

        trial_data_file = data_path.joinpath('trial_data.csv')
        trial_data = pd.read_csv(trial_data_file)

        bdata_file = data_path.joinpath('behaviour_data.pickle')
        with open(bdata_file, 'rb') as file:
            b_data = pickle.load(file)

        ndata_file = data_path.joinpath('neural_data.pickle')
        with open(ndata_file, 'rb') as file:
            n_data = pickle.load(file)

        # Rest of your code for loading data
        scanner_fps = 30.
        vr_fps = 1000.

        end_time = n_data['traces'].shape[1]/scanner_fps
        scanner_times = np.arange(0, end_time, 1./scanner_fps)

        position = b_data["position"]
        times = b_data["time"]
        spikes = n_data["deconvolved"]

        spike_times = []
        for s in spikes:
            spike_times.append(scanner_times[s])
            
        # Perform operations for different env settings
        for env in [1, 2, 3]:
            subset = trial_data[trial_data['env_label'] == env]
            norm_pos, sliced_time, sliced_spikes = pc.slice_data(subset, position, times, spike_times)
            spike_positions = [np.interp(s, sliced_time, norm_pos) for s in sliced_spikes]

            # Rest of your code for processing the current session and env setting
            # Perform additional operations
            firing_rate_maps, occupancy = pc.compute_firing_rate_maps(spike_positions, norm_pos)
            spatial_info = pc.calculate_spatial_info(firing_rate_maps, occupancy)

            # NULL
            n_shuff = 20
            null_spatial_info_distr = []

            for n in range(n_shuff):
                shuff_spike_times = pc.shuffle_spikes(spike_times, scanner_times[-1])
                shuff_norm_pos, shuff_sliced_time, shuff_sliced_spikes = pc.slice_data(subset, position, times,
                                                                                        shuff_spike_times)
                shuff_spike_positions = [np.interp(s, shuff_sliced_time, shuff_norm_pos) for s in shuff_sliced_spikes]

                shuff_firing_rate_maps, shuff_occupancy = pc.compute_firing_rate_maps(shuff_spike_positions,
                                                                                    shuff_norm_pos)
                shuff_spatial_info = pc.calculate_spatial_info(shuff_firing_rate_maps, shuff_occupancy)

                for s in shuff_spatial_info:
                    null_spatial_info_distr.append(s)
            
            # Save plots if required
            if save_plots:
                session_output_folder = output_folder.joinpath(f'{animal_name}_{date}')
                session_output_folder.mkdir(exist_ok=True, parents=True)

                # Plotting per env
                #norm_maps = firing_rate_maps / np.max(firing_rate_maps, axis=1)[:, np.newaxis]
                norm_maps = firing_rate_maps / (np.max(firing_rate_maps, axis=1)[:, np.newaxis] + 1e-8) #small epsilon number to avoid /0 error

                plt.figure(figsize=(15, 5))

                # Find the location of the peak firing rate for each cell
                peak_locations = norm_maps.argmax(axis=1)

                # Sort the firing rate maps based on the peak locations
                ix = np.argsort(peak_locations)

                # Plot the sorted firing rate maps
                plt.imshow(norm_maps[ix, :], cmap="inferno", aspect="auto")

                # Set the x-axis label
                plt.xlabel("location (bins)")

                # Set the y-axis label
                plt.ylabel("cell #")

                # Add a colorbar to the plot
                plt.colorbar()
                
                # Save the ratemaps plot
                ratemaps_folder = session_output_folder.joinpath('ratemaps')
                ratemaps_folder.mkdir(exist_ok=True)
                plt.savefig(ratemaps_folder.joinpath(f'{animal_name}_{date}_env_{env}_firing_rate_maps.png'))
                plt.close() # Close figure to save memory

                # Plotting null_spatial_info_distr
                plt.figure(figsize=(5, 5))
                plt.hist(null_spatial_info_distr, density=True, bins=20, label='Null')
                plt.hist(spatial_info, density=True, alpha=0.5, bins=20, label='Experimental')
                sns.despine()
                plt.xlabel('spatial info (bit/spike)')
                plt.ylabel('probability density')
                plt.legend()
                
                # Save the spatial_distr plot
                spatial_info_folder = session_output_folder.joinpath('spatial_info_distr')
                spatial_info_folder.mkdir(exist_ok=True)
                plt.savefig(spatial_info_folder.joinpath(f'{animal_name}_{date}_env_{env}_null_spatial_info_distr_plot.png'))
                plt.close()
                
            # Calculate place cell statistics
            place_cell_th = np.percentile(null_spatial_info_distr, 95)
            n_place_cells = sum(i > place_cell_th for i in spatial_info)
            fraction = n_place_cells / len(spatial_info)

            # Create a DataFrame with the current session's results
            session_results = pd.DataFrame({'Animal': [animal_name],
                                            'Session Date': [date],
                                            'Environment': [env],
                                            'Number of Place Cells': [n_place_cells],
                                            'Fraction': [fraction]})
            
            # Concatenate the session results with the main results_df
            results_df = pd.concat([results_df, session_results], ignore_index=True)
            
    # Return the updated results dataframe
    return results_df

In [20]:
# Loop over animals
for animal_folder, animal_name, session_dates in zip(animal_folders, animal_names, all_session_dates):
    # Process the sessions for the current animal
    results_df = process_sessions(animal_folder, animal_name, session_dates, results_df)

In [21]:
# Print the final results dataframe
print(results_df)

            Animal Session Date Environment Number of Place Cells  Fraction
0     441406_fiano     20230309           1                    34  0.118056
1     441406_fiano     20230309           2                   101  0.350694
2     441406_fiano     20230309           3                   149  0.517361
3     441406_fiano     20230317           1                     7  0.050725
4     441406_fiano     20230317           2                    71  0.514493
5     441406_fiano     20230317           3                   105  0.760870
6     441406_fiano     20230323           1                   222  0.884462
7     441406_fiano     20230323           2                    99  0.396000
8     441406_fiano     20230323           3                   226  0.904000
9     441406_fiano     20230328           1                   121  0.683616
10    441406_fiano     20230328           2                   119  0.676136
11    441406_fiano     20230328           3                   142  0.793296
12    441406

In [22]:
# Specify the desired path and filename
df_output_path = Path('../data/final/place_cells.csv')

# Export the DataFrame to the specified path and filename
results_df.to_csv(df_output_path, index=False)

*Boxplot by place cell fraction*

In [ ]:
output_folder = Path('./imgs')
output_folder.mkdir(exist_ok=True,parents=True)

save_folder = output_folder.joinpath('place_cell_distribution')
save_folder.mkdir(exist_ok=True, parents=True)

# Read the CSV file into a DataFrame
file_path = Path('../data/final/place_cells.csv')
df = pd.read_csv(file_path)

# Convert the 'Fraction' column to percentages
df['Fraction'] = df['Fraction'] * 100

# Create the boxplot using seaborn with 'Fraction' column as 'y'
sns.boxplot(x='Environment', y='Fraction', data=df)

# Set the labels for the x and y axes
plt.xlabel('Environment')
plt.ylabel('Fraction of Place Cells (%)')

# Calculate the statistical values for each environment
statistics = df.groupby('Environment')['Fraction'].describe()

# Display the statistics as a table
print(statistics)
# Save the statistics as a CSV file at the desired location
save_path = Path('../data/final/place_cell_boxplot_statistics.csv')
statistics.to_csv(save_path)

# Save the plot
plt.savefig(save_folder.joinpath('place_cell_distribution.png'))

# Display the plot
plt.show()

# Sort the DataFrame based on the 'Environment' column
df_sorted = df.sort_values('Environment')

# Display the sorted dataframe
print(df_sorted)